In [7]:
# Run then restart runtime
!apt-get install python3-rtree
!pip install pystac
!pip install geopandas
!pip install rasterio
!pip install descartes
!pip install solaris==0.2.0
!pip install rio-tiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [8]:
!pip install solaris==0.2.0

In [0]:
# Imports
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
import solaris as sol
from pathlib import Path
import rasterio
from rasterio.windows import Window
import geopandas as gpd
from pystac import (Catalog, CatalogType, Item, Asset, LabelItem, Collection)
from rasterio.transform import from_bounds
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
from rio_tiler import main as rt_main
import skimage
from tqdm import tqdm
import os
os.environ["CURL_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt" #Q! chưa rõ để làm gì, dường như liên quan đến đặt môi trường để chạy được requests khi gọi lib requests https://stackoverflow.com/questions/48391750/disable-python-requests-ssl-validation-for-an-imported-module

# We have to add this wrkaround for stackio:
# (https://pystac.readthedocs.io/en/latest/concepts.html#using-stac-io)
from urllib.parse import urlparse
import requests
from pystac import STAC_IO
def my_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)
STAC_IO.read_text_method = my_read_method

In [10]:
!pip install rasterio

In [0]:
# Folder Setup
data_dir = Path('drive/My Drive/open_cities_compet/data/')
data_dir.mkdir(exist_ok=True)

img_path = data_dir/'train/images-256'
mask_path = data_dir/'train/masks-256'
img_path.mkdir(exist_ok=True)
mask_path.mkdir(exist_ok=True)

In [0]:
#Mounted by hand --> khong can mount lại nữa
# from google.colab import drive
# drive.mount('/content/drive')

# Loading the data

In [0]:
def save_tile_img(tif_url, xyz, tile_size, save_path='', prefix='', display=False):
  x,y,z = xyz
  tile, mask = rt_main.tile(tif_url, x,y,z, tilesize=tile_size)
  if display: 
    plt.imshow(np.moveaxis(tile,0,2))
    plt.show()
  skimage.io.imsave(f'{save_path}/{prefix}{z}_{x}_{y}.png',np.moveaxis(tile,0,2), check_contrast=False) 

def save_tile_mask(labels_poly, tile_poly, xyz, tile_size, save_path='', prefix='', display=False, greyscale=True):
  x,y,z = xyz
  tfm = from_bounds(*tile_poly.bounds, tile_size, tile_size) 
  
  cropped_polys = [poly for poly in labels_poly if poly.intersects(tile_poly)]
  cropped_polys_gdf = gpd.GeoDataFrame(geometry=cropped_polys, crs='epsg:4326')
  
  fbc_mask = sol.vector.mask.df_to_px_mask(df=cropped_polys_gdf,
                                         channels=['footprint', 'boundary'], # 'contact'],
                                         affine_obj=tfm, shape=(tile_size,tile_size),
                                         boundary_width=5, boundary_type='inner', contact_spacing=5, meters=True)
  
  if display: plt.imshow(fbc_mask); plt.show()
  
  if greyscale:
    skimage.io.imsave(f'{save_path}/{prefix}{z}_{x}_{y}_mask.png',fbc_mask[:,:,0], check_contrast=False) 
  else:
    skimage.io.imsave(f'{save_path}/{prefix}{z}_{x}_{y}_mask.png',fbc_mask, check_contrast=False) 

def save_area_id_images(area = 'acc', img_id = 'ca041a', label_id='ca041a-labels', zoom_level = 19, tile_size = 256):

  # The item
  one_item = cols[area].get_item(id=img_id)

  # Load labels shapefile
  lab = cols[area].get_item(id=label_id)
  gdf = gpd.read_file(lab.make_asset_hrefs_absolute().assets['labels'].href)
  # get the geometries from the geodataframe
  all_polys = gdf.geometry

  # Get outlines as polygons
  polygon_geom = Polygon(one_item.to_dict()['geometry']['coordinates'][0])
  polygon = gpd.GeoDataFrame(index=[0], crs=gdf.crs, geometry=[polygon_geom])   

  # Tile at zoom_level
  polygon['geometry'].to_file(img_id+'.geojson', driver='GeoJSON')
  !cat {img_id}.geojson | supermercado burn {zoom_level} | mercantile shapes | fio collect > {img_id}{zoom_level}tiles.geojson

  # Load tiles
  tiles = gpd.read_file(f'{img_id}{zoom_level}tiles.geojson')

  # Add a convenience column
  tiles['xyz'] = tiles.id.apply(lambda x: x.lstrip('(,)').rstrip('(,)').split(','))
  tiles['xyz'] = [[int(q) for q in p] for p in tiles['xyz']]

  # IMG URL
  tif_url = one_item.assets['image'].href

  # Sometimes it's just ./id.tif - add full path (should maybe use make_asset_hrefs_absolute instead!!)
  if tif_url.startswith("./"):
    tif_url = '/'.join(one_item.to_dict()['links'][1]['href'].split("/")[:-1])+tif_url[1:]

  print("TIF URL:", tif_url)

  print("Number of tiles:", len(tiles))

  # Loop through tiles, downloading and saving
  for idx in range(len(tiles)):
    tile, mask = rt_main.tile(tif_url, *tiles.iloc[idx]['xyz'], tilesize=tile_size)

    tile_poly = tiles.iloc[idx]['geometry']

    # get affine transformation matrix for this tile using rasterio.transform.from_bounds: https://rasterio.readthedocs.io/en/stable/api/rasterio.transform.html#rasterio.transform.from_bounds
    tfm = from_bounds(*tile_poly.bounds, tile_size, tile_size) 

    # crop geometries to what overlaps our tile polygon bounds
    cropped_polys = [poly for poly in all_polys if poly.intersects(tile_poly)]
    cropped_polys_gdf = gpd.GeoDataFrame(geometry=cropped_polys, crs='epsg:4326')

    # burn a footprint/boundary/contact 3-channel mask with solaris: https://solaris.readthedocs.io/en/latest/tutorials/notebooks/api_masks_tutorial.html
    fbc_mask = sol.vector.mask.df_to_px_mask(df=cropped_polys_gdf,
                                            channels=['footprint', 'boundary'],# 'contact'],
                                            affine_obj=tfm, shape=(tile_size,tile_size),
                                            boundary_width=5, boundary_type='inner', contact_spacing=5, meters=True)

    save_tile_img(tif_url, tiles.iloc[idx]['xyz'], tile_size, save_path=img_path, prefix=f'{area}_{img_id}_{idx}_', display=False)
    save_tile_mask(all_polys, tile_poly, tiles.iloc[idx]['xyz'], tile_size, save_path=mask_path,prefix=f'{area}_{img_id}_{idx}_', display=False)
    print("Saved", f'{area}_{img_id}_{idx}_')

In [0]:
# load our training and test catalogs
train1_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_1/catalog.json')
train2_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_2/catalog.json')
test_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/test/catalog.json')


In [15]:
# Get a list of the possible areas ('scenes) and ids
cols = {cols.id:cols for cols in train1_cat.get_children()}
areas = []
for c in cols:
  itms = [x for x in cols[c].get_all_items()]
  for i, id in enumerate(itms):
    if i % 2 == 0 and i+1 < len(itms):
      areas.append((c, itms[i].id, itms[i+1].id))
print(areas)

[('acc', '665946', '665946-labels'), ('acc', 'a42435', 'a42435-labels'), ('acc', 'ca041a', 'ca041a-labels'), ('acc', 'd41d81', 'd41d81-labels'), ('mon', '401175', '401175-labels'), ('mon', '493701', '493701-labels'), ('mon', '207cc7', '207cc7-labels'), ('mon', 'f15272', 'f15272-labels'), ('ptn', 'abe1a3', 'abe1a3-labels'), ('ptn', 'f49f31', 'f49f31-labels'), ('kam', '4e7c7f', '4e7c7f-labels'), ('dar', 'a017f9', 'a017f9-labels'), ('dar', 'b15fce', 'b15fce-labels'), ('dar', '353093', '353093-labels'), ('dar', 'f883a0', 'f883a0-labels'), ('dar', '42f235', '42f235-labels'), ('dar', '0a4c40', '0a4c40-labels'), ('znz', '33cae6', '33cae6-labels'), ('znz', '3b20d4', '3b20d4-labels'), ('znz', '076995', '076995-labels'), ('znz', '75cdfa', '75cdfa-labels'), ('znz', '9b8638', '9b8638-labels'), ('znz', '06f252', '06f252-labels'), ('znz', 'c7415c', 'c7415c-labels'), ('znz', 'aee7fd', 'aee7fd-labels'), ('znz', '3f8360', '3f8360-labels'), ('znz', '425403', '425403-labels'), ('znz', 'bd5c14', 'bd5c14-l

In [16]:
# Get a list of the possible areas ('scenes) and ids
cols2 = {cols.id:cols for cols in train2_cat.get_children()}
areas2 = []
for c in cols2:
  itms = [x for x in cols2[c].get_all_items()]
  for i, id in enumerate(itms):
    if i % 2 == 0 and i+1 < len(itms):
      areas2.append((c, itms[i].id, itms[i+1].id))
print(areas2)

[('dar', '8737a8', '8737a8-labels'), ('dar', 'e14d1d', 'e14d1d-labels'), ('dar', '8d7dd4', '8d7dd4-labels'), ('dar', '9870ba', '9870ba-labels'), ('dar', '3b3e53', '3b3e53-labels'), ('dar', '0ccd08', '0ccd08-labels'), ('dar', 'ab32c9', 'ab32c9-labels'), ('dar', '94a004', '94a004-labels'), ('dar', 'fb4c1a', 'fb4c1a-labels'), ('dar', 'ca3445', 'ca3445-labels'), ('dar', '82a1f3', '82a1f3-labels'), ('dar', 'cf83de', 'cf83de-labels'), ('dar', '97ce35', '97ce35-labels'), ('dar', '24a7d8', '24a7d8-labels'), ('dar', 'b8faa3', 'b8faa3-labels'), ('dar', '63c3f9', '63c3f9-labels'), ('dar', 'c533fa', 'c533fa-labels'), ('dar', '5fe6fb', '5fe6fb-labels'), ('dar', '385a0e', '385a0e-labels'), ('dar', '759e34', '759e34-labels'), ('dar', 'ef8f27', 'ef8f27-labels'), ('dar', '1d8af6', '1d8af6-labels'), ('dar', 'd2f2f4', 'd2f2f4-labels'), ('dar', 'eadfa3', 'eadfa3-labels'), ('dar', 'cbf72d', 'cbf72d-labels'), ('dar', '5fadcd', '5fadcd-labels'), ('dar', '56e713', '56e713-labels'), ('dar', '219237', '219237-l

In [0]:
# Get a list of the possible areas ('scenes) and ids
# cols3 = {cols.id:cols for cols in test_cat.get_children()}
# areas3 = []
# for c in cols2:
#   itms = [x for x in cols[c].get_all_items()]
#   for i, id in enumerate(itms):
#     if i % 2 == 0 and i+1 < len(itms):
#       areas2.append((c, itms[i].id, itms[i+1].id))
# print(areas2)

In [0]:
completed_task = [('acc', '665946', '665946-labels'), 
                  ('acc', 'a42435', 'a42435-labels'), 
                  ('acc', 'ca041a', 'ca041a-labels'),
                  ('acc', 'd41d81', 'd41d81-labels'),
                  ('mon', '401175', '401175-labels'),
                  ('mon', '493701', '493701-labels'),
                  ('mon', '207cc7', '207cc7-labels'),
                  ('mon', 'f15272', 'f15272-labels'),
                  ('ptn', 'abe1a3', 'abe1a3-labels'),
                  ('ptn', 'f49f31', 'f49f31-labels'),
                  ('kam', '4e7c7f', '4e7c7f-labels'),
                  ('dar', 'a017f9', 'a017f9-labels'),
                  ('dar', 'b15fce', 'b15fce-labels'),
                  ('dar', '353093', '353093-labels'), 
                  ('dar', 'f883a0', 'f883a0-labels'), #bỏ qua vì chạy bị lỗi
                  ('dar', '42f235', '42f235-labels') # bỏ qua vì chạy lỗi
                , ('dar', '0a4c40', '0a4c40-labels')
                  ]

In [19]:
areas

[('acc', '665946', '665946-labels'),
 ('acc', 'a42435', 'a42435-labels'),
 ('acc', 'ca041a', 'ca041a-labels'),
 ('acc', 'd41d81', 'd41d81-labels'),
 ('mon', '401175', '401175-labels'),
 ('mon', '493701', '493701-labels'),
 ('mon', '207cc7', '207cc7-labels'),
 ('mon', 'f15272', 'f15272-labels'),
 ('ptn', 'abe1a3', 'abe1a3-labels'),
 ('ptn', 'f49f31', 'f49f31-labels'),
 ('kam', '4e7c7f', '4e7c7f-labels'),
 ('dar', 'a017f9', 'a017f9-labels'),
 ('dar', 'b15fce', 'b15fce-labels'),
 ('dar', '353093', '353093-labels'),
 ('dar', 'f883a0', 'f883a0-labels'),
 ('dar', '42f235', '42f235-labels'),
 ('dar', '0a4c40', '0a4c40-labels'),
 ('znz', '33cae6', '33cae6-labels'),
 ('znz', '3b20d4', '3b20d4-labels'),
 ('znz', '076995', '076995-labels'),
 ('znz', '75cdfa', '75cdfa-labels'),
 ('znz', '9b8638', '9b8638-labels'),
 ('znz', '06f252', '06f252-labels'),
 ('znz', 'c7415c', 'c7415c-labels'),
 ('znz', 'aee7fd', 'aee7fd-labels'),
 ('znz', '3f8360', '3f8360-labels'),
 ('znz', '425403', '425403-labels'),
 

In [0]:
# You could fetch all the data with:
for a in [a for a in areas if a not in completed_task] :
  save_area_id_images(area = a[0], img_id = a[1], label_id=a[2], zoom_level = 19, tile_size = 256)
  completed_task.append(a)
  print('-'*10+'completed'+'-'*10,'\n', completed_task, '\n'+'-'*25)

TIF URL: https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_1/dar/0a4c40/0a4c40.tif
Number of tiles: 1657
Saved dar_0a4c40_0_
Saved dar_0a4c40_1_
Saved dar_0a4c40_2_
Saved dar_0a4c40_3_
Saved dar_0a4c40_4_
Saved dar_0a4c40_5_
Saved dar_0a4c40_6_
Saved dar_0a4c40_7_
Saved dar_0a4c40_8_
Saved dar_0a4c40_9_
Saved dar_0a4c40_10_
Saved dar_0a4c40_11_
Saved dar_0a4c40_12_
Saved dar_0a4c40_13_
Saved dar_0a4c40_14_
Saved dar_0a4c40_15_
Saved dar_0a4c40_16_
Saved dar_0a4c40_17_
Saved dar_0a4c40_18_
Saved dar_0a4c40_19_
Saved dar_0a4c40_20_
Saved dar_0a4c40_21_
Saved dar_0a4c40_22_
Saved dar_0a4c40_23_
Saved dar_0a4c40_24_
Saved dar_0a4c40_25_
Saved dar_0a4c40_26_
Saved dar_0a4c40_27_
Saved dar_0a4c40_28_
Saved dar_0a4c40_29_
Saved dar_0a4c40_30_
Saved dar_0a4c40_31_
Saved dar_0a4c40_32_
Saved dar_0a4c40_33_
Saved dar_0a4c40_34_
Saved dar_0a4c40_35_
Saved dar_0a4c40_36_
Saved dar_0a4c40_37_
Saved dar_0a4c40_38_
Saved dar_0a4c40_39_
Saved dar_0a4c40_40_
Sav

In [0]:
completed_task2 = []

In [0]:
# You could fetch all the data with:
for a in [a for a in areas2 if a not in completed_task2] :
  save_area_id_images(area = a[0], img_id = a[1], label_id=a[2], zoom_level = 19, tile_size = 256)
  completed_task.append(a)
  print('-'*10+'completed'+'-'*10,'\n', completed_task2, '\n'+'-'*25)

('dar', 'f883a0', 'f883a0-labels')